In [2]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [81]:
import torch
from sklearn import preprocessing
import itertools

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [113]:
def perplex_models_data(models,datasets):
    combos= list(itertools.product(models, datasets))
    print("There are " + str(len(list(combos))) + " combinations of models and datasets:")
    print(combos)
    loadedmodels=[]
    for m,d in combos:
        if m in loadedmodels:
            #figure out how to do this
            print(str(m) + " was already loaded!")
        str(m)+"_tokenizer" = AutoTokenizer.from_pretrained(m)
        str(m)+ "_model" = AutoModelForMaskedLM.from_pretrained(m)
        
        print(str(m) + " model loaded!")
        test = load_dataset(d, split='train', streaming=True)
        print(str(d) + " dataset loaded!")
        data_head = test.take(5000)
        text= [l['text'] for l in list(data_head)]
        encodings = tokenizer('\n\n'.join(text), return_tensors='pt')
        max_length = model.config.max_position_embeddings
        #From https://huggingface.co/transformers/perplexity.html
        stride = 512
        lls = []
        for i in range(0, encodings.input_ids.size(1), stride):
            begin_loc = max(i + stride - max_length, 0)
            end_loc = min(i + stride, encodings.input_ids.size(1))
            trg_len = end_loc - i    # may be different from stride on last loop
            input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
            target_ids = input_ids.clone()
            target_ids[:,:-trg_len] = -100

            with torch.no_grad():
                outputs = model(input_ids, labels=target_ids)
                log_likelihood = outputs[0] * trg_len

            lls.append(log_likelihood)

        ppl = torch.exp(torch.stack(lls).sum() / end_loc)
        print("The perplexity of the " + str(m) + " model with the " + str(d) + " dataset is " + str(ppl.item()))

In [114]:
perplex_models_data(["bert-base-uncased"], ["sentiment140"])

There are 1 combinations of models and datasets:
[('bert-base-uncased', 'sentiment140')]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert-base-uncased model loaded!
sentiment140 dataset loaded!


Token indices sequence length is longer than the specified maximum sequence length for this model (103137 > 512). Running this sequence through the model will result in indexing errors


The perplexity of the bert-base-uncased model with the sentiment140 dataset is 1.7406984567642212


In [ ]:
first 50 from openwebtext: tensor(2.0917)
wiki sample: tensor(1.5268)
500 lines of poetry (poem_sentiment) : tensor(3.2618)
500 tweets from sentiment140: tensor(1.6752)

In [ ]:
from datasets import load_dataset
base_url = "https://storage.googleapis.com/huggingface-nlp/cache/datasets/wikipedia/20200501.en/1.0.0/"
data_files = {"train": base_url + "wikipedia-train.parquet"}
wiki = load_dataset("parquet", data_files=data_files, split="train", streaming=True)
print(next(iter(wiki)))
# {'title': 'Yangliuqing', 'text': 'Yangliuqing () is a market town in Xiqing District...'}